In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import os
import tensorflow as tf , numpy as np

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")
    
    

No GPU found


In [ ]:
tf.reset_default_graph()
def loss_func(G , X , batch_size , index = 0 ):  
    G2 = G[index]
    X2        = tf.transpose(tf.nn.embedding_lookup(tf.transpose(X), index))    
    n1 ,n2    = batch_size , batch_size
    data1     = tf.contrib.framework.sort(G2, axis = 0 )
    data2     = tf.contrib.framework.sort(X2 , axis = 0 )
    data_all  = tf.concat([data1,data2] , axis = 0)
    cdf1      = tf.searchsorted(data1,data_all,side='right' ) / ( batch_size)
    cdf2      = tf.searchsorted(data2,data_all,side='right')/ ( batch_size)
    #d         = tf.reduce_max(tf.abs(cdf1-cdf2))
    d         = tf.reduce_mean(tf.square(data1-data2))
    return tf.to_float(d)

X_dim = 45
z_dim = 30
h_dim = 45
batch_size = 500
X = tf.placeholder(tf.float32, shape=[None, X_dim], name="X")
z = tf.placeholder(tf.float32, shape=[None, z_dim], name="z")
D_W1 = tf.get_variable("DW1",[z_dim, h_dim], dtype=tf.float32)
slice_output = tf.matmul(z , D_W1)
slice_0 = tf.transpose(tf.nn.embedding_lookup(tf.transpose(slice_output), 0))
slice_1 = tf.transpose(tf.nn.embedding_lookup(tf.transpose(slice_output), 1))
slice_output = [slice_0 , slice_1]

loss1 = loss_func(slice_output , X , batch_size , index = 0 )
loss2 = loss_func(slice_output , X , batch_size , index = 1 )
loss  = loss1 + loss2
# sess = tf.InteractiveSession()
X11 = np.random.normal(size= [1000,45])
Z11 = np.random.normal(size= [1000,30])
# H = np.random.normal(size= [1000,45])
# H =[tf.transpose(tf.nn.embedding_lookup(tf.transpose(H), 0))]
# index = 0
# mb_size = 500
# X11 = np.random.normal(size= [1000,45])
# H2 = loss_func(H ,X11 , mb_size , index)
# print(H2.eval())


opt = tf.train.GradientDescentOptimizer(0.0001).minimize(loss)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(10000):
        _ , lss = sess.run([opt, loss] , feed_dict={z: Z11 , X : X11})
        if i % 1000  == 0:
            print(lss)


/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


0.039914772
0.03305521
0.028434543
0.025310863
0.023182351
0.021708496
0.020659985
0.019896008
0.019325906
